In [1]:
import os

In [2]:
from keras.preprocessing import image

In [3]:
import cv2

In [4]:
categories = ['with_mask', 'without_mask']

In [5]:
data = []
for category in categories:
    path = os.path.join('train', category)
    
    label = categories.index(category)
    
    for file in os.listdir(path):
        
        img_path = os.path.join(path,file)
        img = cv2.imread(img_path)
        img = cv2.resize(img,(224,224))
        
        data.append([img,label])

In [6]:
len(data)

1508

In [8]:
import random

In [9]:
random.shuffle(data)

In [10]:
X = []
y = []

for features,label in data:
    X.append(features)
    y.append(label)

In [11]:
len(X)

1508

In [12]:
len(y)

1508

In [13]:
import numpy as np

In [14]:
X = np.array(X)
y = np.array(y)

In [15]:
X.shape

(1508, 224, 224, 3)

In [16]:
y.shape

(1508,)

In [17]:
y

array([0, 1, 0, ..., 1, 1, 1])

In [18]:
X = X/255

In [19]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)

In [20]:
X_train.shape

(1206, 224, 224, 3)

In [21]:
X_test.shape

(302, 224, 224, 3)

In [22]:
from keras.applications.vgg16 import VGG16

In [23]:
vgg = VGG16()

In [24]:
vgg.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [25]:
from keras import Sequential

In [26]:
model = Sequential()

In [27]:
for layer in vgg.layers[:-1]:
    model.add(layer)

In [28]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       2

In [29]:
for layer in model.layers:
    layer.trainable=False

In [30]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       2

In [31]:
from keras.layers import Dense

In [32]:
model.add(Dense(1,activation='sigmoid'))

In [33]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       2

In [67]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
cv = CountVectorizer()
tfidf = TfidfVectorizer(max_features=3000)

In [53]:
model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])

In [54]:
model.fit(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Epoch 1/5
38/38 [==============================] - 264s 7s/step - loss: 0.5389 - accuracy: 0.7388 - val_loss: 0.3869 - val_accuracy: 0.9073
Epoch 2/5
38/38 [==============================] - 260s 7s/step - loss: 0.3128 - accuracy: 0.9129 - val_loss: 0.2813 - val_accuracy: 0.9404
Epoch 3/5
38/38 [==============================] - 259s 7s/step - loss: 0.2399 - accuracy: 0.9328 - val_loss: 0.2454 - val_accuracy: 0.9404
Epoch 4/5
38/38 [==============================] - 252s 7s/step - loss: 0.2013 - accuracy: 0.9411 - val_loss: 0.2511 - val_accuracy: 0.8974
Epoch 5/5
38/38 [==============================] - 256s 7s/step - loss: 0.1807 - accuracy: 0.9486 - val_loss: 0.1717 - val_accuracy: 0.9669


In [66]:
cap = cv2.VideoCapture(0)

In [71]:
while True:
    
    ret, frame = cap.read()
    
    # call the detection method
    img = cv2.resize(frame,(224,224))
    
    y_pred = detect_face_mask(img)
    
    coods = detect_face(cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY))
    
    for x,y,w,h in coods:
        
        cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),3)
    
    if y_pred == 0:
        draw_label(frame,"MASK DETECTED",(30,30),(0,255,0))
    else:
        draw_label(frame,"NO MASK DETECTED",(30,30),(0,0,255))
        
    
    
    print(y_pred)
    
    cv2.imshow("window", frame)
    
    if cv2.waitKey(1) & 0xFF == ord('x'):
        break
    
cv2.destroyAllWindows()

error: OpenCV(4.4.0) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-h4wtvo23\opencv\modules\imgproc\src\resize.cpp:3929: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'


In [56]:
def detect_face_mask(img):
    y_pred = model.predict_classes(img.reshape(1,224,224,3))
    
    return y_pred[0][0]

In [38]:
sample = cv2.imread('sample1.jpg')
sample = cv2.resize(sample,(224,224))

error: OpenCV(4.4.0) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-h4wtvo23\opencv\modules\imgproc\src\resize.cpp:3929: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'


In [41]:
detect_face_mask(sample)

NameError: name 'sample1' is not defined

In [57]:
def draw_label(img,text,pos,bg_color):
    
    text_size = cv2.getTextSize(text,cv2.FONT_HERSHEY_SIMPLEX,1,cv2.FILLED)
    
    end_x = pos[0] + text_size[0][0] + 2
    end_y = pos[1] + text_size[0][1] - 2
    
    cv2.rectangle(img,pos,(end_x,end_y),bg_color,cv2.FILLED)
    cv2.putText(img,text,pos,cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,0),1,cv2.LINE_AA)

In [60]:
while True:
    
    ret, frame = cap.read()
    
    coods = detect_face(cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY))
    
    for x,y,w,h in coods:
        
        cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),3)
    
    
    cv2.imshow("window", frame)
    
    if cv2.waitKey(1) & 0xFF == ord('x'):
        break
        
cv2.destroyAllWindows()

In [58]:
haar = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [59]:
def detect_face(img):
    
    coods = haar.detectMultiScale(img)
    
    return coods

In [70]:
import pickle
pickle.dump(tfidf,open('vectorizer.pkl','wb'))
pickle.dump(y_pred,open('model.pkl','wb'))